<a href="https://colab.research.google.com/github/kk890528/python-work/blob/master/Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

###下載cifar10資料集

In [0]:
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data() 

###資料整理

In [53]:
X_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [0]:
# Reshape size
#X_train = X_train.reshape(-1, 32*32,3)
#X_test = X_test.reshape(-1, 32*32,3)
#將資料轉成0到1之間
X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

# One-hot encoding
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

In [55]:
X_train.shape

(50000, 32, 32, 3)

### Functional API 的操作方式練習
####用cnn做


In [0]:
#CNN Layers
f_1 = Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_1')
f_2 =  MaxPool2D()
f_3= Conv2D(256, (3, 3), padding='same', activation='relu', name='Conv_2')
f_4=MaxPool2D()
f_5=Conv2D(512, (3, 3), padding='same', activation='relu', name='Conv_3')
f_6=GlobalAveragePooling2D()
#Fnn layers
f_7=Dense(552, activation='relu')
f_8 =Dense(10, activation='softmax')
x = Input(shape=(32,32,3))

In [57]:
print(x)

Tensor("input_11:0", shape=(None, 32, 32, 3), dtype=float32)


In [0]:
h_1 = f_1(x)
h_2 = f_2(h_1)
h_3= f_3(h_2)
h_4= f_4(h_3)
h_5= f_5(h_4)
h_6= f_6(h_5)
h_7= f_7(h_6)
y = f_8(h_7)

In [59]:
print(y)

Tensor("dense_30/Identity:0", shape=(None, 10), dtype=float32)


In [60]:
model = Model(x, y)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 8, 8, 512)         1180160   
_________________________________________________________________
global_average_pooling2d_18  (None, 512)               0   

###訓練看看

In [0]:
model.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [62]:
 model.fit(X_train,Y_train, 
           batch_size=128, 
           epochs=10,
          validation_data=(X_test, Y_test)
           )

Epoch 1/10
391/391 [==============================] - 9s 24ms/step - loss: 1.7767 - categorical_accuracy: 0.3249 - val_loss: 1.4891 - val_categorical_accuracy: 0.4424
Epoch 2/10
391/391 [==============================] - 9s 24ms/step - loss: 1.3456 - categorical_accuracy: 0.5043 - val_loss: 1.2257 - val_categorical_accuracy: 0.5535
Epoch 3/10
391/391 [==============================] - 9s 24ms/step - loss: 1.1460 - categorical_accuracy: 0.5853 - val_loss: 1.0593 - val_categorical_accuracy: 0.6271
Epoch 4/10
391/391 [==============================] - 9s 24ms/step - loss: 1.0254 - categorical_accuracy: 0.6330 - val_loss: 0.9797 - val_categorical_accuracy: 0.6427
Epoch 5/10
391/391 [==============================] - 9s 24ms/step - loss: 0.9428 - categorical_accuracy: 0.6648 - val_loss: 0.9525 - val_categorical_accuracy: 0.6558
Epoch 6/10
391/391 [==============================] - 9s 24ms/step - loss: 0.8627 - categorical_accuracy: 0.6932 - val_loss: 0.9705 - val_categorical_accuracy: 0.653

In [63]:
score = model.evaluate(X_train, Y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

5/5 [==============================] - 3s 675ms/step - loss: 0.5869 - categorical_accuracy: 0.7930
Loss: 0.5869226455688477
準確率: 79.29999828338623



##嘗試看看
###非線性堆疊

In [0]:
from tensorflow.keras.layers import concatenate, add

## 先嘗試在FNN的地方進行分岔

In [0]:
f_1 = Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_1')
f_2 =  MaxPool2D()
f_3= Conv2D(256, (3, 3), padding='same', activation='relu', name='Conv_2')
f_4=MaxPool2D()
f_5=Conv2D(512, (3, 3), padding='same', activation='relu', name='Conv_3')
f_6=GlobalAveragePooling2D()
f_7=Dense(552, activation='relu')
###新增f_a
f_a=Dense(817, activation='relu')
f_8 =Dense(10, activation='softmax')
x = Input(shape=(32,32,3))

In [0]:
h_1 = f_1(x)
h_2 = f_2(h_1)
h_3= f_3(h_2)
h_4= f_4(h_3)
h_5= f_5(h_4)
h_6= f_6(h_5)
h_7= f_7(h_6)
z=f_a(h_6)
u = concatenate([h_7, z])
y = f_8(u)

In [67]:
model2 = Model(x, y)
model2.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 32, 32, 128)  3584        input_12[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_34 (MaxPooling2D) (None, 16, 16, 128)  0           Conv_1[0][0]                     
__________________________________________________________________________________________________
Conv_2 (Conv2D)                 (None, 16, 16, 256)  295168      max_pooling2d_34[0][0]           
____________________________________________________________________________________________

In [0]:
model2.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [69]:
 model2.fit(X_train,Y_train, 
           batch_size=128, 
           epochs=10,
          validation_data=(X_test, Y_test)
           )

Epoch 1/10
391/391 [==============================] - 9s 24ms/step - loss: 1.7135 - categorical_accuracy: 0.3526 - val_loss: 1.4104 - val_categorical_accuracy: 0.4840
Epoch 2/10
391/391 [==============================] - 9s 24ms/step - loss: 1.2738 - categorical_accuracy: 0.5325 - val_loss: 1.1735 - val_categorical_accuracy: 0.5795
Epoch 3/10
391/391 [==============================] - 9s 24ms/step - loss: 1.0972 - categorical_accuracy: 0.6027 - val_loss: 1.0944 - val_categorical_accuracy: 0.6066
Epoch 4/10
391/391 [==============================] - 9s 24ms/step - loss: 0.9730 - categorical_accuracy: 0.6541 - val_loss: 0.9652 - val_categorical_accuracy: 0.6577
Epoch 5/10
391/391 [==============================] - 9s 24ms/step - loss: 0.8816 - categorical_accuracy: 0.6871 - val_loss: 0.9845 - val_categorical_accuracy: 0.6486
Epoch 6/10
391/391 [==============================] - 9s 24ms/step - loss: 0.8095 - categorical_accuracy: 0.7167 - val_loss: 0.7859 - val_categorical_accuracy: 0.726

In [70]:
score = model2.evaluate(X_train, Y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

5/5 [==============================] - 3s 678ms/step - loss: 0.5165 - categorical_accuracy: 0.8201
Loss: 0.5165384411811829
準確率: 82.00600147247314


#嘗試看看在CNN的地方非線性(設兩個CNN)

In [0]:
#CNN1
f_1 = Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_1')
f_2 =  MaxPool2D()
f_3= Conv2D(256, (3, 3), padding='same', activation='relu', name='Conv_2')
f_4=MaxPool2D()
f_5=Conv2D(512, (3, 3), padding='same', activation='relu', name='Conv_3')
f_6=GlobalAveragePooling2D()
f_7=Dense(552, activation='relu')
#CNN2
f_a1 = Conv2D(50, (4, 4), padding='same', activation='relu', name='Conv_a1')
f_a2 =  MaxPool2D()
f_a3= Conv2D(100, (4, 4), padding='same', activation='relu', name='Conv_a2')
f_a4=MaxPool2D()
f_a5=Conv2D(200, (4, 4), padding='same', activation='relu', name='Conv_a3')
f_a6=GlobalAveragePooling2D()
f_a7=Dense(817, activation='relu')
#FNN
f_8 =Dense(10, activation='softmax')
x = Input(shape=(32,32,3))

In [0]:
#cnn1
h_1 = f_1(x)
h_2 = f_2(h_1)
h_3= f_3(h_2)
h_4= f_4(h_3)
h_5= f_5(h_4)
h_6= f_6(h_5)
h_7= f_7(h_6)
#cnn2
h_a1 = f_a1(x)
h_a2 = f_a2(h_a1)
h_a3= f_a3(h_a2)
h_a4= f_a4(h_a3)
h_a5= f_a5(h_a4)
h_a6= f_a6(h_a5)
h_a7= f_a7(h_a6)
#合體
u = concatenate([h_7, h_a7])
y = f_8(u)

In [73]:
model3= Model(x, y)
model3.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 32, 32, 128)  3584        input_13[0][0]                   
__________________________________________________________________________________________________
Conv_a1 (Conv2D)                (None, 32, 32, 50)   2450        input_13[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_36 (MaxPooling2D) (None, 16, 16, 128)  0           Conv_1[0][0]                     
____________________________________________________________________________________________

In [0]:
model3.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [75]:
 model3.fit(X_train,Y_train, 
           batch_size=128, 
           epochs=10,
          validation_data=(X_test, Y_test)
           )

Epoch 1/10
391/391 [==============================] - 15s 39ms/step - loss: 1.7030 - categorical_accuracy: 0.3617 - val_loss: 1.4515 - val_categorical_accuracy: 0.4651
Epoch 2/10
391/391 [==============================] - 15s 39ms/step - loss: 1.3015 - categorical_accuracy: 0.5233 - val_loss: 1.2289 - val_categorical_accuracy: 0.5546
Epoch 3/10
391/391 [==============================] - 15s 39ms/step - loss: 1.1279 - categorical_accuracy: 0.5930 - val_loss: 1.1320 - val_categorical_accuracy: 0.5895
Epoch 4/10
391/391 [==============================] - 15s 39ms/step - loss: 0.9956 - categorical_accuracy: 0.6444 - val_loss: 0.9183 - val_categorical_accuracy: 0.6751
Epoch 5/10
391/391 [==============================] - 15s 39ms/step - loss: 0.8912 - categorical_accuracy: 0.6819 - val_loss: 0.9457 - val_categorical_accuracy: 0.6634
Epoch 6/10
391/391 [==============================] - 15s 39ms/step - loss: 0.8098 - categorical_accuracy: 0.7134 - val_loss: 0.8094 - val_categorical_accuracy:

In [76]:
score = model3.evaluate(X_train, Y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

5/5 [==============================] - 5s 988ms/step - loss: 0.5259 - categorical_accuracy: 0.8140
Loss: 0.5258731245994568
準確率: 81.40199780464172


### 三個model比較

In [79]:
model.summary()
print('分隔線是我------ \n')
model2.summary()
print('分隔線是我------ \n')
model3.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 8, 8, 512)         1180160   
_________________________________________________________________
global_average_pooling2d_18  (None, 512)               0   